In [30]:
import torch
from vtk import *
from vtk.util import numpy_support
import numpy as np
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [28]:

def compute_PSNR(arrgt,arr_recon):
    try:
        diff = arrgt - arr_recon
        sqd_max_diff = (np.max(arrgt)-np.min(arrgt))**2
        if(np.mean(diff**2) == 0):
            raise ZeroDivisionError("dividing by zero, cannot calculate psnr")
        snr = 10*np.log10(sqd_max_diff/np.mean(diff**2))
        return snr
    except ZeroDivisionError as err:
        return str(err)



def read_vti(filename):
    reader = vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()
    return reader.GetOutput()


def writeVti(data, filename):
    writer = vtkXMLImageDataWriter()
    writer.SetFileName(filename)
    writer.SetInputData(data)
    writer.Write()


def createVtkImageData(origin, dimensions, spacing):
    localDataset = vtkImageData()
    localDataset.SetOrigin(origin)
    localDataset.SetDimensions(dimensions)
    localDataset.SetSpacing(spacing)
    return localDataset

In [17]:
directory_path = '/Users/manasvijain/Desktop/Autoencoder/matrices' 
files = [f"Matrix_{i}.txt" for i in range(50)]
matrices = []

for file in files:
    file_path = os.path.join(directory_path, file) 
    matrix = np.loadtxt(file_path)  
    if matrix.shape != (250, 250):
        raise ValueError(f"Expected matrix size 250x250, but got {matrix.shape} in file {file}")
    matrices.append(matrix)  

data = np.stack(matrices, axis=2)
data = data.astype(np.float32)
# data = (data - data.min()) / (data.max() - data.min())
print(data.shape)  # Should be (250, 250, 50)
print(data.dtype)
four_d_tensor = torch.from_numpy(data).float() 
input_tensor = four_d_tensor.unsqueeze(0).unsqueeze(0)
print("Corrected input tensor shape:", input_tensor.shape)

(250, 250, 50)
float32
Corrected input tensor shape: torch.Size([1, 1, 250, 250, 50])


In [29]:
vtifile = read_vti('test.vti')
data = numpy_support.vtk_to_numpy(vtifile)
print(data.shape)  # Should be (250, 250, 50)
print(data.dtype)

2024-12-13 18:16:01.585 (71346.789s) [           9963E]       vtkXMLParser.cxx:368    ERR| vtkXMLDataParser (0x1112e3020): Error parsing XML in stream at line 1, column 0, byte index 0: not well-formed (invalid token)
2024-12-13 18:16:01.586 (71346.790s) [           9963E]       vtkXMLReader.cxx:576    ERR| vtkXMLImageDataReader (0x1166f2da0): Error parsing input file.  ReadXMLInformation aborting.
2024-12-13 18:16:01.586 (71346.790s) [           9963E]       vtkExecutive.cxx:730    ERR| vtkCompositeDataPipeline (0x112c6f050): Algorithm vtkXMLImageDataReader (0x1166f2da0) returned failure for request: vtkInformation (0x111220600)
  Debug: Off
  Modified Time: 286
  Reference Count: 1
  Registered Events: (none)
  Request: REQUEST_INFORMATION
  ALGORITHM_AFTER_FORWARD: 1
  FORWARD_DIRECTION: 0




AttributeError: 'ImageData' object has no attribute 'GetDataType'

In [ ]:
dataset = TensorDataset(input_tensor, input_tensor) 
loader = DataLoader(dataset, batch_size=1)

In [26]:
import torch
import torch.nn as nn

class Conv3DAutoencoder(nn.Module):
    def __init__(self):
        super(Conv3DAutoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv3d(1, 16, kernel_size=3, stride=2, padding=1),  # Output: (16, 125, 125, 25)
            nn.ReLU(),
            nn.BatchNorm3d(16),
            nn.Conv3d(16, 32, kernel_size=3, stride=2, padding=1), # Output: (32, 63, 63, 13)
            nn.ReLU(),
            nn.BatchNorm3d(32),
            nn.Conv3d(32, 64, kernel_size=3, stride=2, padding=1), # Output: (64, 32, 32, 7)
            nn.ReLU(),
            nn.BatchNorm3d(64),
            nn.Conv3d(64, 128, kernel_size=3, stride=2, padding=1), # Output: (128, 16, 16, 4)
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(128 * 16 * 16 * 4, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 512),
            nn.ReLU()
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 128 * 16 * 16 * 4),
            nn.ReLU(),
            nn.Unflatten(1, (128, 16, 16, 4)),
            nn.ConvTranspose3d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=(1,1,0)), # Match dimension to 32, 32, 8
            nn.ReLU(),
            nn.ConvTranspose3d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=0), # Match dimension to 63, 63, 15
            nn.ReLU(),
            nn.ConvTranspose3d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=0), # Match dimension to 126, 126, 30
            nn.ReLU(),
            nn.ConvTranspose3d(16, 1, kernel_size=3, stride=2, padding=1, output_padding=(1, 1, 1)), # Match dimension to 250, 250, 59
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize the model
model = Conv3DAutoencoder()
model.eval()

# Create a dummy input tensor
dummy_input = torch.rand(1, 1, 250, 250, 50)

# Forward pass the dummy input through the model
with torch.no_grad():
    output = model(dummy_input)

# Print the shape of the output
print("Output shape:", output.shape)


Output shape: torch.Size([1, 1, 250, 250, 50])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Conv3DAutoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 50  # Set the number of epochs

for epoch in range(num_epochs):
    for data, target in loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

In [ ]:
model.eval()
with torch.no_grad():
    for data, _ in loader:
        data = data.to(device)
        latent = model.encoder(data)
        print("Latent Shape:", latent.shape)  

In [ ]:
print(latent)

NameError: name 'latent' is not defined